In [42]:
!pip install pandas matplotlib dash


# 6. Data Preparation for KPI's and ML

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

crm_ml = pd.read_csv('data/crm_anomaly_df.csv')
erp_ml = pd.read_csv('data/erp_anomaly_df.csv')


# Merge CRM and ERP datasets on 'customer_id' and 'order_id'
merged_df = pd.merge(crm_ml, erp_ml, on=['customer_id', 'order_id'])


In [44]:
import pandas as pd

# Assuming crm_ml and erp_ml are your CRM and ERP DataFrames
merged_df = pd.merge(crm_ml, erp_ml, on=['customer_id', 'order_id'])

# Sample 5000 random rows from the merged dataset
sampled_df = merged_df.sample(n=5000, random_state=42)


# 6.1 Creating dataset suitable for ML models

In [45]:
# List of columns to remove
columns_to_remove = ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 
                     'geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 
                     'order_id', 'order_item_id', 'product_id', 'seller_id','customer_city', 
                     'customer_state', 'geolocation_city','geolocation_state', 'seller_city'
                     ,'seller_state', 'product_category_name','payment_type']

# Removing the columns from the DataFrame
sampled_df_ml = sampled_df.drop(columns=columns_to_remove)

In [46]:
sampled_df_ml['order_status'].value_counts()

delivered    4999
canceled        1
Name: order_status, dtype: int64

In [47]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()

# Convert 'order_status' to numerical values
sampled_df_ml['order_status'] = le.fit_transform(sampled_df_ml['order_status'])

# Check the conversion
print(sampled_df_ml['order_status'].value_counts())

1    4999
0       1
Name: order_status, dtype: int64


In [48]:
# Adding a new column shipping_time
merged_df['order_purchase_timestamp'] = pd.to_datetime(merged_df['order_purchase_timestamp'])
merged_df['order_delivered_customer_date'] = pd.to_datetime(merged_df['order_delivered_customer_date'])

# Now, calculate shipping time
sampled_df_ml['shipping_time'] = (merged_df['order_delivered_customer_date'] - merged_df['order_purchase_timestamp']).dt.days



In [49]:
# Check the data types of each column
print(sampled_df_ml.dtypes)

# Check if all columns are numeric
is_numeric = merged_df.apply(lambda col: pd.api.types.is_numeric_dtype(col)).all()
print("Is the entire dataset numeric?", is_numeric)


review_score                       int64
anomaly_z_score_x                  int64
anomaly_isolation_forest_x         int64
anomaly_dbscan_x                   int64
shipping_limit_date               object
price                            float64
freight_value                    float64
order_status                       int64
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
payment_sequential               float64
payment_installments             float64
payment_value                    float64
product_name_lenght              float64
product_description_lenght       float64
product_photos_qty               float64
product_weight_g                 float64
product_length_cm                float64
product_height_cm                float64
product_width_cm                 float64
seller_zip_code_prefix             int64
anomaly_z_score_

In [50]:
import pandas as pd

# Convert columns to datetime format (if not already)
datetime_columns = [
    'shipping_limit_date',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'order_purchase_timestamp'
]

for column in datetime_columns:
    sampled_df_ml[column] = pd.to_datetime(sampled_df_ml[column], errors='coerce')

# Function to extract date features
def extract_date_features(df, column_name):
    df[f'{column_name}_year'] = df[column_name].dt.year
    df[f'{column_name}_month'] = df[column_name].dt.month
    df[f'{column_name}_day'] = df[column_name].dt.day
    df[f'{column_name}_weekday'] = df[column_name].dt.weekday
    df[f'{column_name}_hour'] = df[column_name].dt.hour

# Apply the function to each datetime column
for column in datetime_columns:
    extract_date_features(sampled_df_ml, column)

# Drop the original datetime columns
sampled_df_ml.drop(columns=datetime_columns, inplace=True)

print(sampled_df_ml.dtypes)




review_score                               int64
anomaly_z_score_x                          int64
anomaly_isolation_forest_x                 int64
anomaly_dbscan_x                           int64
price                                    float64
freight_value                            float64
order_status                               int64
payment_sequential                       float64
payment_installments                     float64
payment_value                            float64
product_name_lenght                      float64
product_description_lenght               float64
product_photos_qty                       float64
product_weight_g                         float64
product_length_cm                        float64
product_height_cm                        float64
product_width_cm                         float64
seller_zip_code_prefix                     int64
anomaly_z_score_y                          int64
anomaly_isolation_forest_y                 int64
anomaly_dbscan_y    

1. Random Forest Regressor for Customer Lifetime Value (CLV)¶¶

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Take a random sample of the dataset for testing
merged_df_sample = sampled_df_ml.sample(n=5000, random_state=42)

# Assuming 'shipping_time' is a proxy for CLV
X_clv = merged_df_sample.drop(['shipping_time'], axis=1)
y_clv = merged_df_sample['shipping_time']

# Check for missing values
print(X_clv.isnull().sum())

# Optionally drop rows with missing values
X_clv = X_clv.dropna()
y_clv = y_clv[X_clv.index]  # Align y with the new X

# Split data
X_clv_train, X_clv_test, y_clv_train, y_clv_test = train_test_split(X_clv, y_clv, test_size=0.05, random_state=42)

# Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_clv_train, y_clv_train)

# Predictions
y_clv_pred = rf_model.predict(X_clv_test)

# Evaluation
mse_clv = mean_squared_error(y_clv_test, y_clv_pred)
print(f"Random Forest Regressor MSE for CLV: {mse_clv}")


review_score                             0
anomaly_z_score_x                        0
anomaly_isolation_forest_x               0
anomaly_dbscan_x                         0
price                                    0
freight_value                            0
order_status                             0
payment_sequential                       0
payment_installments                     0
payment_value                            0
product_name_lenght                      0
product_description_lenght               0
product_photos_qty                       0
product_weight_g                         0
product_length_cm                        0
product_height_cm                        0
product_width_cm                         0
seller_zip_code_prefix                   0
anomaly_z_score_y                        0
anomaly_isolation_forest_y               0
anomaly_dbscan_y                         0
shipping_limit_date_year                 0
shipping_limit_date_month                0
shipping_li

In [53]:
print(merged_df_sample.columns)


Index(['review_score', 'anomaly_z_score_x', 'anomaly_isolation_forest_x',
       'anomaly_dbscan_x', 'price', 'freight_value', 'order_status',
       'payment_sequential', 'payment_installments', 'payment_value',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix',
       'anomaly_z_score_y', 'anomaly_isolation_forest_y', 'anomaly_dbscan_y',
       'shipping_time', 'shipping_limit_date_year',
       'shipping_limit_date_month', 'shipping_limit_date_day',
       'shipping_limit_date_weekday', 'shipping_limit_date_hour',
       'order_approved_at_year', 'order_approved_at_month',
       'order_approved_at_day', 'order_approved_at_weekday',
       'order_approved_at_hour', 'order_delivered_carrier_date_year',
       'order_delivered_carrier_date_month',
       'order_delivered_carrier_date_day',
       'order_delivered_carrier_date_weekday'

 2. Gradient Boosting Machines (GBM) for Sales Forecasting

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming 'price' is the target for sales forecasting
X_sales = sampled_df_ml.drop(['price'], axis=1)
y_sales = sampled_df_ml['price']

# Sample a smaller portion of the dataset (e.g., 10% of the data)
X_sales_sampled = X_sales.sample(frac=0.1, random_state=42)
y_sales_sampled = y_sales.sample(frac=0.1, random_state=42)

# Split data
X_sales_train, X_sales_test, y_sales_train, y_sales_test = train_test_split(X_sales_sampled, y_sales_sampled, 
                                                                            test_size=0.05, random_state=42)

# Option 1: HistGradientBoostingRegressor (no need for n_jobs)
hgb_model = HistGradientBoostingRegressor(max_iter=50, random_state=42)  # Reduced iterations for faster processing
hgb_model.fit(X_sales_train, y_sales_train)

# Predictions
y_sales_pred_hgb = hgb_model.predict(X_sales_test)

# Evaluation
mse_sales_hgb = mean_squared_error(y_sales_test, y_sales_pred_hgb)
print(f"Hist Gradient Boosting MSE for Sales Forecasting: {mse_sales_hgb}")

# Option 2: Linear Regression (simpler and faster model)
lr_model = LinearRegression()
lr_model.fit(X_sales_train, y_sales_train)

# Predictions
y_sales_pred_lr = lr_model.predict(X_sales_test)

# Evaluation
mse_sales_lr = mean_squared_error(y_sales_test, y_sales_pred_lr)
print(f"Linear Regression MSE for Sales Forecasting: {mse_sales_lr}")


Hist Gradient Boosting MSE for Sales Forecasting: 409.54246884463123
Linear Regression MSE for Sales Forecasting: 3114.946906823991


3. Support Vector Machines (SVM) for Customer Churn

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Sample data (replace with your actual dataset)
df = sampled_df_ml.copy()

# Step 1: Handle datetime columns
datetime_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

# Convert datetime columns to numeric (e.g., days since epoch)
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col]).astype(int) / 10**9  # Convert to seconds since epoch

# Step 2: Handle categorical variables
df = pd.get_dummies(df, drop_first=True)

# Step 3: Check for missing values and drop them
df = df.dropna()

# Step 4: Separate features (X) and target (y)
X_churn = df.drop(['anomaly_z_score_y'], axis=1)
y_churn = df['anomaly_z_score_y']

# Step 5: Reduce dataset by sampling (e.g., 1% of the data for testing)
X_churn_sampled = X_churn.sample(frac=0.01, random_state=42)
y_churn_sampled = y_churn.loc[X_churn_sampled.index]  # Align y with sampled X

# Step 6: Standardize the features
scaler = StandardScaler()
X_churn_scaled = scaler.fit_transform(X_churn_sampled)

# Step 7: Split the data into training and testing sets
X_churn_train, X_churn_test, y_churn_train, y_churn_test = train_test_split(
    X_churn_scaled, y_churn_sampled, test_size=0.2, random_state=42
)

# Step 8: Train the Linear SVC model
svm_model = LinearSVC(max_iter=1000, random_state=42)
svm_model.fit(X_churn_train, y_churn_train)

# Step 9: Make predictions
y_churn_pred_svm = svm_model.predict(X_churn_test)

# Step 10: Evaluate the model
accuracy_churn_svm = accuracy_score(y_churn_test, y_churn_pred_svm)
print(f"SVM Accuracy for Customer Churn: {accuracy_churn_svm}")


SVM Accuracy for Customer Churn: 1.0


 4. K-Nearest Neighbors (KNN) for Product Recommendations

In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Assuming merged_df_ml is already loaded and contains the necessary columns
df = sampled_df_ml.copy()

# Step 1: Handle datetime columns
datetime_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

# Convert datetime columns to numeric (e.g., seconds since epoch)
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').astype(int) / 10**9  # Handle invalid dates with 'coerce'

# Step 2: Handle categorical variables
df = pd.get_dummies(df, drop_first=True)

# Step 3: Check for missing values and drop rows with NaNs
df = df.dropna()

# Step 4: Separate features (X) and target (y)
X_knn = df.drop(['anomaly_z_score_y'], axis=1)
y_knn = df['anomaly_z_score_y']

# Step 5: Reduce dataset by sampling (e.g., 1% of the data for faster processing)
X_knn_sampled = X_knn.sample(frac=0.01, random_state=42)
y_knn_sampled = y_knn.loc[X_knn_sampled.index]  # Align target with sampled features

# Step 6: Standardize the data
scaler = StandardScaler()
X_knn_scaled = scaler.fit_transform(X_knn_sampled)

# Step 7: Split the data into training and testing sets
X_knn_train, X_knn_test, y_knn_train, y_knn_test = train_test_split(
    X_knn_scaled, y_knn_sampled, test_size=0.05, random_state=42)

# Step 8: Train the KNN Classifier
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_knn_train, y_knn_train)

# Step 9: Make predictions
y_knn_pred = knn_model.predict(X_knn_test)

# Step 10: Evaluate the model
accuracy_knn = accuracy_score(y_knn_test, y_knn_pred)
print(f"KNN Accuracy for Recommendations: {accuracy_knn}")



KNN Accuracy for Recommendations: 0.6666666666666666


/Users/omkacham/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



 5. Linear Regression for Average Order Value (AOV)

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Step 1: Create a copy of the dataset for modifications
df = sampled_df_ml.copy()

# Step 2: Handle datetime columns
datetime_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

# Convert datetime columns to numeric (e.g., seconds since epoch)
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').astype(int) / 10**9  # Handle invalid dates with 'coerce'

# Step 3: Handle categorical variables
df = pd.get_dummies(df, drop_first=True)

# Step 4: Check for missing values and drop rows with NaNs
df = df.dropna()

# Step 5: Separate features (X) and target (y)
X_aov = df.drop(['price'], axis=1)  # Features
y_aov = df['price']  # Target variable

# Step 6: Reduce dataset by sampling (taking 1% of the data for faster processing)
X_aov_sampled = X_aov.sample(frac=0.01, random_state=42)
y_aov_sampled = y_aov.loc[X_aov_sampled.index]  # Align target with sampled features

# Step 7: Standardize the data
scaler = StandardScaler()
X_aov_scaled = scaler.fit_transform(X_aov_sampled)

# Step 8: Split data into training and testing sets
X_aov_train, X_aov_test, y_aov_train, y_aov_test = train_test_split(
    X_aov_scaled, y_aov_sampled, test_size=0.05, random_state=42)

# Step 9: Train the Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_aov_train, y_aov_train)

# Step 10: Make predictions
y_aov_pred = lr_model.predict(X_aov_test)

# Step 11: Evaluate the model
mse_aov = mean_squared_error(y_aov_test, y_aov_pred)
print(f"Linear Regression Mean Squared Error (MSE) for AOV: {mse_aov}")


Linear Regression Mean Squared Error (MSE) for AOV: 47840.04714497432


# 7. Automated Business Intelligence (BI) Dashboard

In [116]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import pandas as pd


# Assume `sampled_df` is already created and cleaned with proper datetime formatting
sampled_df['order_purchase_timestamp'] = pd.to_datetime(sampled_df['order_purchase_timestamp'])
sampled_df['order_delivered_customer_date'] = pd.to_datetime(sampled_df['order_delivered_customer_date'])
sampled_df['order_estimated_delivery_date'] = pd.to_datetime(sampled_df['order_estimated_delivery_date'])


In [117]:
# Now assuming `sampled_df` is already created and cleaned

# Category 1: Customer KPIs
customer_kpis = {
    'Total Customers': sampled_df['customer_id'].nunique(),
    'Retention Rate (%)': (sampled_df[sampled_df['order_id'].duplicated(keep=False)]['customer_unique_id'].nunique() / sampled_df['customer_id'].nunique()) * 100,
    'New Customers': sampled_df[sampled_df['order_purchase_timestamp'] >= '2023-01-01']['customer_unique_id'].nunique(),
    'Churn Rate (%)': 100 - ((sampled_df[sampled_df['order_id'].duplicated(keep=False)]['customer_unique_id'].nunique() / sampled_df['customer_id'].nunique()) * 100)
}

# Category 2: Review and Anomaly KPIs
review_anomaly_kpis = {
    'Average Review Score': sampled_df['review_score'].mean(),
    'Anomaly Rate (%)': (sampled_df[sampled_df['anomaly_z_score_x'] == 1].shape[0] / sampled_df.shape[0]) * 100,
    'Review Count': sampled_df['review_score'].nunique(),
    'Positive Review Rate (%)': (sampled_df[sampled_df['review_score'] >= 4].shape[0] / sampled_df.shape[0]) * 100
}

# Category 3: Shipping and Delivery KPIs
shipping_delivery_kpis = {
    'Avg Shipping Time in days': (sampled_df['order_delivered_customer_date'] - sampled_df['order_purchase_timestamp']).dt.days.mean(),
    'Shipping Cost per Order': sampled_df['freight_value'].mean(),
    'Late Deliveries (%)': (sampled_df[sampled_df['order_delivered_customer_date'] > sampled_df['order_estimated_delivery_date']].shape[0] / sampled_df.shape[0]) * 100,
    'Delivered Orders': sampled_df[sampled_df['order_status'] == 'delivered'].shape[0]
}

# Category 4: Financial KPIs
financial_kpis = {
    'Average Order Value (AOV)': sampled_df['price'].mean(),
    'Total Revenue': sampled_df['price'].sum(),
    'Average Payment Value': sampled_df['payment_value'].mean(),
    'Installment Usage Rate (%)': (sampled_df[sampled_df['payment_installments'] > 1].shape[0] / sampled_df.shape[0]) * 100
}


# Machine Leanring 
random_forest_kpis = {
    'Random Forest Regressor MSE for CLV': mse_clv,
    'Linear Regression MSE for Sales Forecasting': mse_sales_lr,
    'SVM Accuracy for Customer Churn': accuracy_churn_svm,
    'KNN Accuracy for Recommendations': accuracy_knn,
    'Linear Regression Mean Squared Error (MSE) for AOV': mse_aov
}


In [132]:
# Visualizations (Pie Charts and Bar Graphs)

payment_method_usage = sampled_df['payment_type'].value_counts(normalize=True) * 100
payment_pie = {
    'data': [
        {
            'values': payment_method_usage.values,
            'labels': payment_method_usage.index,
            'type': 'pie',
            'name': 'Payment Methods',
            'marker': {'colors': ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99']}
        }
    ],
    'layout': {
        'title': 'Payment Method Usage',
        'font': {'color': 'black'},
        'height': 600,
        'width': 600
    }
}

review_score_distribution = sampled_df['review_score'].value_counts(normalize=True) * 100
review_pie = {
    'data': [
        {
            'values': review_score_distribution.values,
            'labels': review_score_distribution.index,
            'type': 'pie',
            'name': 'Review Scores',
            'marker': {'colors': ['#FF6666', '#FFCC66', '#66FF66', '#66CCFF', '#9966FF']}
        }
    ],
    'layout': {
        'title': 'Review Score Distribution',
        'font': {'color': 'black'},
        'height': 600,
        'width': 600
    }
}

# Bar Chart: Customer Location Distribution
customer_location_distribution = sampled_df['customer_city'].value_counts()
location_bar = {
    'data': [
        {
            'x': customer_location_distribution.index,
            'y': customer_location_distribution.values,
            'type': 'bar',
            'name': 'Customer Locations',
            'marker': {'color': '#66B2FF'}
        }
    ],
    'layout': {
        'title': 'Customer Location Distribution',
        'font': {'color': 'black'},
        'xaxis': {'color': 'black',
                  'tickangle': 45  # Optional: Adjust the angle of the tick labels to avoid overlap
                 },
        'yaxis': {'title': 'Number of Customers', 'color': 'black', 'range': [0, 450]},
        'height': 600,
        'width': 900
    }
}


# Bar Chart: Top Selling Products
top_selling_products = sampled_df['product_category_name'].value_counts()
products_bar = {
    'data': [
        {
            'x': top_selling_products.index,
            'y': top_selling_products.values,
            'type': 'bar',
            'name': 'Top Products',
            'marker': {'color': '#FF9999'}
        }
    ],
    'layout': {
        'title': 'Top Selling Products',
        'font': {'color': 'black'},
        'xaxis': { 'color': 'black',
                  'tickangle': 45, 'tickfont': {'size': 10}   # Optional: Adjust the angle of the tick labels to avoid overlap
                 },
        'yaxis': {'title': 'Number Sold', 'color': 'black'},
        'height':600,
        'width': 900
    }
}


In [144]:
# Defining the Dash app

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div(children=[
    html.H1("Automated Business Intelligence (BI) Dashboard", style={'text-align': 'center', 'color': 'black'}),


    html.Div([
        # First Section: KPIs (Top 4x4 grid)
        html.Div([
            # Customer KPIs
             #html.H3('Customer KPIs', style={'color': 'black', 'text-align': 'center','font-size': '15px'}),
            *[
                html.Div([
                    html.H4(kpi, style={'color': 'black', 
                                        'font-size': '10px',
                                        'white-space': 'nowrap',       # Prevent the text from wrapping to the next line
                                        'text-overflow': 'ellipsis'}),  # Set font-size for customer_kpis titles only
                    
                    html.P(f'{value:.2f}' if isinstance(value, float) else value, style={'color': 'black'})  # Keep default font-size for values
                ], style={'border': '3px solid #ccc',   # Border around the box
                          'padding': '8px',            # Inner padding (space between content and border)
                          'margin': '8px',             # Outer margin (space between the box and surrounding elements)
                          'border-radius': '25px',     # Border radius to control the roundness of corners
                          'text-align': 'center',       # Center the text inside the box
                          'width': '180px',             # Set a fixed width for the box
                          'height': '110px',            # Set a fixed height for the box (optional)
                          'box-sizing': 'border-box'})
                for kpi, value in customer_kpis.items()
            ],
            
            # Review and Anomaly KPIs 
             #html.H3('Review and Anomaly KPIs', style={'color': 'black', 'text-align': 'center','font-size': '15px'}),
            *[
                html.Div([
                    html.H4(kpi, style={'color': 'black', 
                                        'font-size': '10px',
                                        'white-space': 'nowrap',       # Prevent the text from wrapping to the next line
                                        'text-overflow': 'ellipsis'}),  # Set font-size for review_anomaly_kpis titles only
                    
                    html.P(f'{value:.2f}' if isinstance(value, float) else value, style={'color': 'black'})  # Keep default font-size for values
                ], style={'border': '3px solid #ccc',   # Border around the box
                          'padding': '8px',            # Inner padding (space between content and border)
                          'margin': '8px',             # Outer margin (space between the box and surrounding elements)
                          'border-radius': '25px',     # Border radius to control the roundness of corners
                          'text-align': 'center',       # Center the text inside the box
                          'width': '180px',             # Set a fixed width for the box
                          'height': '110px',            # Set a fixed height for the box (optional)
                          'box-sizing': 'border-box'})
                for kpi, value in review_anomaly_kpis.items()
            ],

            # Shipping and Delivery KPIs 
             #html.H3('Shipping and Delivery KPI', style={'color': 'black', 'text-align': 'center','font-size': '15px'}),
            *[
                html.Div([
                    html.H4(kpi, style={'color': 'black', 
                                        'font-size': '10px',
                                        'white-space': 'nowrap',       # Prevent the text from wrapping to the next line
                                        'text-overflow': 'ellipsis'}),  # Set font-size for shipping_delivery_kpis titles only
                    
                    html.P(f'{value:.2f}' if isinstance(value, float) else value, style={'color': 'black'})  # Keep default font-size for values
                ], style={'border': '3px solid #ccc',   # Border around the box
                          'padding': '8px',            # Inner padding (space between content and border)
                          'margin': '8px',             # Outer margin (space between the box and surrounding elements)
                          'border-radius': '25px',     # Border radius to control the roundness of corners
                          'text-align': 'center',       # Center the text inside the box
                          'width': '180px',             # Set a fixed width for the box
                          'height': '110px',            # Set a fixed height for the box (optional)
                          'box-sizing': 'border-box'})
                for kpi, value in shipping_delivery_kpis.items()
            ],
            
            
            # Financial KPIs
            #html.H3('Financial KPIs', style={'color': 'black', 'text-align': 'center','font-size': '15px'}),
            *[
                html.Div([
                    html.H4(kpi, style={'color': 'black', 
                                        'font-size': '10px',
                                        'white-space': 'nowrap',       # Prevent the text from wrapping to the next line
                                        'text-overflow': 'ellipsis'}),  # Set font-size for financial_kpis titles only
                    
                    html.P(f'{value:.2f}' if isinstance(value, float) else value, style={'color': 'black'})  # Keep default font-size for values
                ], style={'border': '3px solid #ccc',   # Border around the box
                          'padding': '8px',            # Inner padding (space between content and border)
                          'margin': '8px',             # Outer margin (space between the box and surrounding elements)
                          'border-radius': '25px',     # Border radius to control the roundness of corners
                          'text-align': 'center',       # Center the text inside the box
                          'width': '180px',             # Set a fixed width for the box
                          'height': '110px',            # Set a fixed height for the box (optional)
                          'box-sizing': 'border-box'})
                for kpi, value in financial_kpis.items()   
            ],
            
            
            # Machine Leanring 
            #html.H3('Machine Learning KPIs', style={'color': 'black', 'text-align': 'center','font-size': '15px'}),
            *[
                html.Div([
                    html.H4(kpi, style={
                        'color': 'black', 
                        'font-size': '10px',
                        'text-overflow': 'ellipsis'}),
                    html.P(f'{value:.2f}' if isinstance(value, float) else value, style={'color': 'black'})
                ], style={
                    'border': '3px solid #ccc',
                    'padding': '8px',
                    'margin': '8px',
                    'border-radius': '25px',
                    'text-align': 'center',
                    'width': '180px',
                    'height': '110px',
                    'box-sizing': 'border-box'
                }) 
                for kpi, value in random_forest_kpis.items()
            ]                        
         
        ], 
            
            
            
            style={'display': 'grid',
                  'grid-template-columns': 'repeat(4, 1fr)',  # 4 equal-width columns
                  'gap': '20px',              # This sets both row and column gap
                  'grid-row-gap': '100px',     # Specific control over row gap
                  'grid-column-gap': '5px',  # Specific control over column gap
                  'width': '50%',            # Set the width of the container
                  'height': '50%',
                  'padding': '1px'}),
        
        
        
        

        # Second Section: Pie Charts (Up and Down) 
        
        html.Div
        ([
            dcc.Graph(figure=payment_pie, style={'width': '100%', 'height': '50%'}),
            dcc.Graph(figure=review_pie, style={'width': '100%', 'height': '50%'})
        ], 
            style={'width': '25%', 'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'}),

        # Third Section: Bar Graphs (Up and Down)
        html.Div([
            dcc.Graph(figure=location_bar, style={'width': '100%', 'height': '50%'}),
            dcc.Graph(figure=products_bar, style={'width': '100%', 'height': '50%'})
        ], style={'width': '41%', 'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
    ], style={'display': 'flex', 'width': '100%'})
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
#http://127.0.0.1:8050